# Visuliazation of some paramaters of robot
- this jupyter notbook show the variables related to the robot


## import

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
import ipywidgets as widgets
from rt2_assignment1.srv import Command
from IPython.display import display
from tf import transformations
import actionlib
import actionlib.msg
import collections
import math

rospy.init_node('visualized_node')

## display the movemennt of the robot

# visualize the trajectory of the robot
- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity) 
https://python.atelierkobato.com/annotate/
https://www.it-swarm-ja.com/ja/python/python-matplotlib%E3%82%A2%E3%83%8B%E3%83%A1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%81%A7%E5%81%9C%E6%AD%A2%E9%96%8B%E5%A7%8B%E4%B8%80%E6%99%82%E5%81%9C%E6%AD%A2/1073484243/

- an xy graph showing the robot’s position and the orientation 

In [2]:
# First set up the figure and the axis 
fig, ax = plt.subplots()

ax.set_xlim(( -10, 10))
ax.set_ylim((-10, 10))

line, = ax.plot([], [], 'ro')
x_pos=0
y_pos=0
x_data=collections.deque(maxlen=10)
y_data=collections.deque(maxlen=10)
yaw=0
theta=collections.deque(maxlen=10)
arrow = ax.quiver(1,1,1,1, alpha = 0)

def init():
    line.set_data([], [])
    return (line,)


def odom_callback(msg):
    #global x_data,y_data,yaw,theta
    x_pos=msg.pose.pose.position.x
    print(x_pos)
    y_pos=msg.pose.pose.position.y
    y_data.append(y_pos)
    x_data.append(x_pos)

    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    theta.append(yaw)
    

def animate(i):
    #global x_data,y_data,yaw,arrow,theta
    global arrow
    arrow.remove()
    line.set_data(x_data, y_data)
    
#     point = {
#         'start': [x_data[-1], y_data[-1]],
#         'end': [x_data[-1]+x_data[-1]*math.cos(yaw), y_data[-1]+y_data[-1]*math.sin(yaw)]
#     }
#     arrow = plt.Arrow(x=x_pos,y=y_pos,dx=10*np.cos(yaw),dy=10*np.cos(yaw),width=0.01,head_width=0.05,head_length=0.2,length_includes_head=True,color='k')
#     arrow=ax.annotate('', xy=point['end'], xytext=point['start'],
#                 arrowprops=dict(shrink=0, width=1, headwidth=8, 
#                                 headlength=10, connectionstyle='arc3',
#                                 facecolor='gray', edgecolor='gray')
#                )
#     ax.add_patch(arrow)
    x = x_data[-1]
    y = y_data[-1]
    a = np.cos(yaw) 
    b = np.sin(yaw)
    arrow = ax.quiver(x, y, a, b,scale = 0.5, color='k', pivot = 'tail', units='xy', alpha=0.6, edgecolor = 'black')
    
    return (line,arrow)



jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# First set up the figures and the axis 
    # Robot position and orientation
fig, ax = plt.subplots()
ax.set_xlim(( -10, 10))
ax.set_ylim((-10, 10))
ax.set_aspect("equal")
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Robot positon")

line, = ax.plot([], [], 'ro', markersize="5")

x_data =  collections.deque(maxlen=300)
y_data = collections.deque(maxlen=300)
theta_data = collections.deque(maxlen=300)

#quiver initialization
Q = ax.quiver(1,1,1,1, alpha = 0)

def init():
    line.set_data([], [])
    return (line,)
    
def animate(i):
    global Q
    Q.remove()
    
    line.set_data(x_data, y_data)     
    delta = 0.2
    X = x_data[-1] + delta * np.cos(theta_data[-1])
    Y = y_data[-1] + delta * np.sin(theta_data[-1])
    U = delta * np.cos(theta_data[-1]) + np.cos(theta_data[-1])
    V = delta * np.sin(theta_data[-1]) + np.sin(theta_data[-1])
    Q = ax.quiver(X, Y, U, V,scale = 0.5, color='k', pivot = 'tail', units='xy', alpha=0.6, edgecolor = 'gray')
    return (line, Q)

def odom_callback(msg):
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    quaternion = (msg.pose.pose.orientation.x, msg.pose.pose.orientation.y,msg.pose.pose.orientation.z, msg.pose.pose.orientation.w)
    euler_angles = transformations.euler_from_quaternion(quaternion)
    theta_data.append(euler_angles[2]) 
      
                  
# Read robot odometry
jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.)
anim_position = animation.FuncAnimation(fig, animate, init_func=init, blit=True)

ERROR:tornado.application:Exception in callback <bound method TimerBase._on_timer of <matplotlib.backends.backend_webagg_core.TimerTornado object at 0x7f357cc4e490>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/usr/lib/python3/dist-packages/matplotlib/backend_bases.py", line 1194, in _on_timer
    ret = func(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1447, in _step
    still_going = Animation._step(self, *args)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1173, in _step
    self._draw_next_frame(framedata, self._blit)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1192, in _draw_next_frame
    self._draw_frame(framedata)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1755, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "/tmp/ipy

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now


- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity)

In [3]:
fig1, ax1 = plt.subplots()

ax1.set_xlim(( 0, 100))
ax1.set_ylim((-1.2, 1.2))

linear_cmd_vel=[]
angular_cmd_vel=[]
angular_cmd_vel=[]

time=[]

line1, = ax1.plot([], [], 'r-')
line2, = ax1.plot([], [], 'b-')
line3, = ax1.plot([], [], 'k*')
zero_time = rospy.get_time()

def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    return (line1,line2,line3)

def twist_callback(msg):
    global linear_cmd_vel,angular_cmd_vel,zero_time,time
    linear_cmd_vel.append(msg.linear.x)
    angular_cmd_vel.append(msg.angular.z)
    time.append(rospy.get_time()-zero_time)

def animate1(i):
    global linear_cmd_vel,angular_cmd_vel,time,actual_linear_vel
    line1.set_data(time,linear_cmd_vel)
    line2.set_data(time, angular_cmd_vel)
    #line3.set_data(time, actual_linear_vel)

    return (line1,line2,line3)

jr.subscribe('/cmd_vel', Twist, twist_callback)

anim = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

- a boxplot displaying the number of reached targets and cancelled targets


In [5]:
# fig2, ax2 = plt.subplots()

# ax2.set_xlim(( 0, 2))
# ax2.set_ylim((0.0, 10))
# ngoal= 0
# ncancel=0
# goal=0

# def init2():
#     ngoal= rospy.get_param("goal_param")
#     ncancel= rospy.get_param("goal_param")
#     goal=0

# def animate2(i):
#     global ngoal,ncancel,goal
#     index=1
#     goal=goal+1
#     ngoal= rospy.get_param("goal_param")
#     ncancel=rospy.get_param("cancel_param")
#     ax2.bar(index,ngoal,label="the number of goal", align="center", color='blue', width=0.35)
#     ax2.bar(index+0.35, ncancel, align='center', width=0.35,color='r', label='the number of cancel') 

# # def animate2(i):
# #     global ngoal,ncancel,goal
# #     print()
# #     index=1

# anim = animation.FuncAnimation(fig2, animate2,init_func=init2, 
#                                frames=100, interval=20,blit=True)

In [7]:
ngoal=0
ncancel=0
fig2, ax2 = plt.subplots()

ax2.set_xlim(( 0, 2))
ax2.set_ylim((0.0, 10))

def result_callback(msg):
    global ngoal, ncancel
    if msg.result.reached==True:
        ngoal=ngoal+1
        print(ngoal)
    #elif msg.result.reached==False:
    else:
        ncancel=ncancel+1

def animate2(i):
    global ngoal,ncancel
    index=1
    ax2.bar(index,ngoal,label="the number of goal", align="center", color='blue', width=0.35)
    ax2.bar(index+0.35, ncancel, align='center', width=0.35,color='r', label='the number of cancel') 
        
jr.subscribe('/go_to_point/result', rt2_assignment1.msg.MotionActionResult, result_callback)

anim = animation.FuncAnimation(fig2, animate2, 
                               frames=100, interval=20,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now


In [8]:
print(ngoal)

0


- a hist plot showing the time required to reach targets

In [9]:
fig3, ax3 = plt.subplots()
ax3.set_ylim((0.0, 5))
goal_time_array=[]
index=0
def init3():
    print('init')
def reached_callback(msg):
    global goal_time_array
    goal_time_array.append(msg.data)
    print(goal_time_array)
    
def animate3(i):
    global goal_time_array
    ax3.hist(goal_time_array,bins=10, align='mid',ec="yellow", fc="green", alpha=0.5) 
jr.subscribe('/reached_time', Float32 , reached_callback)

anim = animation.FuncAnimation(fig3, animate3,init_func=init3, 
                               frames=100, interval=200,blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
goal_time_array=[]
fig3, ax3 = plt.subplots()
ax3.set_ylim((0.0, 1))

def reached_callback(msg):
    global goal_time_array
    goal_time_array.append(msg.data)
    print(goal_time_array)
    ax3.hist(goal_time_array,bins=15, lw=1,ec="yellow", fc="green", alpha=0.5) 

jr.subscribe('/reached_time', Float32 , reached_callback)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now
